In [1]:
import pyodbc
import pandas as pd
import json

In [2]:
contraseña = open("password.json", "r")
YOUR_PASSWORD = json.load(contraseña) #json es sólo el string de su contraseña.
contraseña.close()

In [5]:
# Set up your connection string
server = 'act-evaluacion-server.database.windows.net'
database = 'act-evaluacion-db'
username = 'Alejandro'
password = YOUR_PASSWORD

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")

Connected to the Azure SQL Database successfully!


In [6]:
query_1 = "SELECT name FROM sys.tables"

cursor = conn.cursor()
cursor.execute(query_1)

tables = list(cursor.fetchall())
print(tables)

[('Customer',), ('ProductModel',), ('ProductDescription',), ('Product',), ('ProductModelProductDescription',), ('ProductCategory',), ('BuildVersion',), ('ErrorLog',), ('Address',), ('CustomerAddress',), ('SalesOrderDetail',), ('SalesOrderHeader',)]


In [7]:
query_2 = "SELECT * FROM SalesLT.customer"
datos = pd.read_sql(query_2, conn)
datos.head()

/var/folders/rb/fqqlv8sj6g176x662s9tq20w0000gn/T/ipykernel_44752/129695103.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datos = pd.read_sql(query_2, conn)


,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01


In [8]:
conn.close()

In [10]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd

In [11]:
fecha_referencia = datetime(2000, 1, 1)
datos['ModifiedDate'] = pd.to_datetime(datos['ModifiedDate'])
datos['DiasDesdeReferencia'] = (datos['ModifiedDate'] - fecha_referencia).dt.days

columnas_a_eliminar = ['PasswordHash', 'PasswordSalt', 'rowguid', 'ModifiedDate']
X = datos.drop(columnas_a_eliminar + ['DiasDesdeReferencia'], axis=1)

y = datos['DiasDesdeReferencia']

columnas_categoricas = X.select_dtypes(include=['object', 'bool']).columns.tolist()
columnas_numericas = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [12]:
if columnas_categoricas:
    for col in columnas_categoricas:
        X[col] = X[col].fillna('Desconocido')
    
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_cats = encoder.fit_transform(X[columnas_categoricas])
    
    encoded_df = pd.DataFrame(
        encoded_cats, 
        columns=encoder.get_feature_names_out(columnas_categoricas)
    )
    
    X = X.drop(columnas_categoricas, axis=1)
    X = pd.concat([X, encoded_df], axis=1)

for col in columnas_numericas:
    X[col] = X[col].fillna(X[col].mean())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
scaler = StandardScaler()
if columnas_numericas:
    X_train[columnas_numericas] = scaler.fit_transform(X_train[columnas_numericas])
    X_test[columnas_numericas] = scaler.transform(X_test[columnas_numericas])

modelo = LinearRegression()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

predicciones_fecha = [fecha_referencia + pd.Timedelta(days=int(pred)) for pred in y_pred]
fechas_reales = [fecha_referencia + pd.Timedelta(days=int(real)) for real in y_test]

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R²: {r2:.4f}")

MAE: 76.84
MSE: 27016.93
R²: 0.7255


In [14]:
def predecir_fecha(nuevo_cliente):
    if columnas_categoricas:
        for col in columnas_categoricas:
            if col in nuevo_cliente.columns:
                nuevo_cliente[col] = nuevo_cliente[col].fillna('Desconocido')
        
        nuevo_encoded = encoder.transform(nuevo_cliente[columnas_categoricas])
        if hasattr(nuevo_encoded, 'toarray'):
            nuevo_encoded = nuevo_encoded.toarray()
            
        nuevo_encoded_df = pd.DataFrame(
            nuevo_encoded,
            columns=encoder.get_feature_names_out(columnas_categoricas)
        )
        
        nuevo_cliente = nuevo_cliente.drop(columnas_categoricas, axis=1)
        nuevo_cliente = pd.concat([nuevo_cliente, nuevo_encoded_df], axis=1)
    
    for col in columnas_numericas:
        if col in nuevo_cliente.columns:
            nuevo_cliente[col] = nuevo_cliente[col].fillna(X[col].mean())
    
    if columnas_numericas:
        nuevo_cliente[columnas_numericas] = scaler.transform(nuevo_cliente[columnas_numericas])
    
    dias_predichos = modelo.predict(nuevo_cliente)[0]
    fecha_predicha = fecha_referencia + pd.Timedelta(days=int(dias_predichos))
    
    print(f"Fecha de modificación predicha: {fecha_predicha.strftime('%Y-%m-%d')}")

In [15]:
# Ejemplo con cliente random
cliente_ejemplo = pd.DataFrame({
    'CustomerID': [1001],
    'NameStyle': [False],
    'Title': ['Sr.'],
    'FirstName': ['Carlos'],
    'MiddleName': [None],
    'LastName': ['Gómez'],
    'Suffix': [None],
    'CompanyName': ['Empresa Nueva'],
    'SalesPerson': ['adventure-works\\david8'],
    'EmailAddress': ['carlos@ejemplo.com'],
    'Phone': ['555-123-4567']
})

predecir_fecha(cliente_ejemplo)

Fecha de modificación predicha: 2006-09-09


In [16]:
from flask import Flask, request, jsonify
import pickle
from datetime import datetime
import pandas as pd
import pyodbc
import json

In [17]:
# Guardar todos los componentes necesarios
pickle.dump(modelo, open('modelo.pkl', 'wb'))
pickle.dump(encoder, open('encoder.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(columnas_categoricas, open('columnas_categoricas.pkl', 'wb'))
pickle.dump(columnas_numericas, open('columnas_numericas.pkl', 'wb'))

print("Modelo y componentes guardados")

Modelo y componentes guardados


In [18]:
from azureml.core import Workspace, Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [23]:
import json


#GETTING MY ID:
id = open('/Users/diegoortega/Documents/VS/Python/Semestre 8/Activdad1CloudComputing/id_confidencial.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]

from azureml.core import Workspace

ws = Workspace.create(name="workspaceAct",
                      subscription_id = my_id,
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="modelo.pkl",
                                  model_name=mname,
                                  workspace=ws)

Deploying KeyVault with name workspackeyvault61192520.
Deploying AppInsights with name workspacinsights15568ba1.
Deployed AppInsights with name workspacinsights15568ba1. Took 7.78 seconds.
Deploying StorageAccount with name workspacstoragef6b437b42.
Deployed KeyVault with name workspackeyvault61192520. Took 24.83 seconds.
Deployed StorageAccount with name workspacstoragef6b437b42. Took 29.99 seconds.
Deploying Workspace with name workspaceAct.
Deployed Workspace with name workspaceAct. Took 41.55 seconds.
Registering model model


In [29]:
import json
from azureml.core import Workspace, Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice

# Cargar tu ID de suscripción
with open('id_confidencial.json', 'r') as f:
    mi = json.load(f)
    subscription_id = mi["my_id"]

# Conectar al workspace
ws = Workspace.get(
    name="workspaceAct",
    subscription_id=subscription_id,
    resource_group="resource_group2"
)

# Definir configuración del entorno
env = Environment.from_pip_requirements(name="customer_model_env", file_path="requirements.txt")

# Definir configuración de inferencia
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

# Definir configuración del servicio web
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=True,
    description="Servicio para predecir fecha de modificación de clientes"
)

# Desplegar el servicio
model = Model(ws, name="model")
service = Model.deploy(
    workspace=ws,
    name="customer-date-prediction",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)

# Esperar a que se complete el despliegue
service.wait_for_deployment(show_output=True)

# Obtener la URL del servicio
print(f"URL del servicio: {service.scoring_uri}")

# Guardar la clave primaria para autenticación
primary_key = service.get_keys()[0] if service.auth_enabled else ""
print(f"Primary key: {primary_key}")

/var/folders/rb/fqqlv8sj6g176x662s9tq20w0000gn/T/ipykernel_44752/771110926.py:37: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-23 19:00:20-06:00 Creating Container Registry if not exists..
2025-04-23 19:10:20-06:00 Registering the environment.
2025-04-23 19:10:21-06:00 Building image..
2025-04-23 19:18:04-06:00 Generating deployment configuration.
2025-04-23 19:18:05-06:00 Submitting deployment to compute..
2025-04-23 19:18:13-06:00 Checking the status of deployment customer-date-prediction..
2025-04-23 19:19:41-06:00 Checking the status of inference endpoint customer-date-prediction.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 5a0b3aea-f422-482f-a684-2a9aeb4fdc6c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: customer-date-prediction. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image a99b76d9244b4e0aa4f2c9c11e31d2f9.azurecr.io/azureml/azureml_661f9b3bc4652abf8226273e5f4c8820@sha256:273dcd96f344cc48e68f36005e31afa431fc3603df09b27f0d2a

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 5a0b3aea-f422-482f-a684-2a9aeb4fdc6c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: customer-date-prediction. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image a99b76d9244b4e0aa4f2c9c11e31d2f9.azurecr.io/azureml/azureml_661f9b3bc4652abf8226273e5f4c8820@sha256: